In [164]:
from dotenv import load_dotenv
import os
import requests

load_dotenv()

API_KEY = os.getenv("API_KEY")
URL = f"https://api.resrobot.se/v2.1/trip?format=json&originId=740000001&destId=740000003&passlist=true&showPassingPoints=true&accessId={API_KEY}"
response = requests.get(URL)
result= response.json()
result.keys() 


dict_keys(['Trip', 'ResultStatus', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'planRtTs', 'requestId', 'scrB', 'scrF'])

In [165]:
len(result["Trip"])

9

In [166]:
result["Trip"][0].keys()

dict_keys(['Origin', 'Destination', 'ServiceDays', 'LegList', 'calculation', 'TripStatus', 'idx', 'tripId', 'ctxRecon', 'duration', 'rtDuration', 'checksum'])

In [167]:
result["Trip"][0]["LegList"]["Leg"][0]["Origin"]["name"]


'Stockholm Centralstation'

In [168]:
result["Trip"][0]["LegList"]["Leg"][0]["Destination"]["name"]   

'Malmö Centralstation'

In [169]:
def get_trips(origin_id=740000001, destination_id=740098001):
    """origing_id and destination_id can be found from Stop lookup API"""
    url = f"https://api.resrobot.se/v2.1/trip?format=json&originId={origin_id}&destId={destination_id}&passlist=true&showPassingPoints=true&accessId={API_KEY}"

    try:
        response = requests.get(url)
        response.raise_for_status()

        return response.json()
    except requests.exceptions.RequestException as err:
        print(f"Network or HTTP error: {err}")


result = get_trips()
result.keys()

dict_keys(['Trip', 'ResultStatus', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'planRtTs', 'requestId', 'scrB', 'scrF'])

In [170]:
# checks first trip keys
example_trip = result["Trip"][0]
example_trip.keys()

dict_keys(['Origin', 'Destination', 'ServiceDays', 'LegList', 'calculation', 'TripStatus', 'idx', 'tripId', 'ctxRecon', 'duration', 'rtDuration', 'checksum'])

In [171]:
example_trip['Origin']

{'name': 'Stockholm Centralstation',
 'type': 'ST',
 'id': 'A=1@O=Stockholm Centralstation@X=18058151@Y=59330136@U=1@L=740000001@',
 'extId': '740000001',
 'lon': 18.058151,
 'lat': 59.330136,
 'routeIdx': 1,
 'prognosisType': 'PROGNOSED',
 'time': '16:26:00',
 'date': '2025-01-07',
 'minimumChangeDuration': 'PT20M'}

In [172]:
example_trip['Destination']

{'name': 'Göteborg Centralstation',
 'type': 'ST',
 'id': 'A=1@O=Göteborg Centralstation@X=11973479@Y=57708895@U=1@L=740000002@',
 'extId': '740000002',
 'lon': 11.973479,
 'lat': 57.708895,
 'routeIdx': 6,
 'prognosisType': 'PROGNOSED',
 'time': '20:00:00',
 'date': '2025-01-07',
 'minimumChangeDuration': 'PT15M'}

In [173]:
example_trip['LegList']['Leg'][0].keys()

dict_keys(['Origin', 'Destination', 'Notes', 'JourneyDetailRef', 'JourneyStatus', 'Product', 'Stops', 'JourneyDetail', 'id', 'idx', 'name', 'number', 'category', 'type', 'reachable', 'waitingState', 'direction', 'directionFlag', 'duration'])

In [174]:
example_stops = example_trip["LegList"]["Leg"][0]["Stops"]["Stop"]
example_stops

[{'name': 'Stockholm Centralstation',
  'id': 'A=1@O=Stockholm Centralstation@X=18058151@Y=59330136@U=1@L=740000001@',
  'extId': '740000001',
  'routeIdx': 1,
  'lon': 18.058151,
  'lat': 59.330136,
  'depTime': '16:26:00',
  'depDate': '2025-01-07',
  'depDir': 'Göteborg Centralstation',
  'minimumChangeDuration': 'PT20M'},
 {'Notes': {'Note': [{'value': 'Stannar endast för påstigande',
     'key': 'text.realtime.stop.exit.disabled',
     'type': 'R',
     'txtN': 'Stannar endast för påstigande'}]},
  'name': 'Flemingsberg station (Huddinge kn)',
  'id': 'A=1@O=Flemingsberg station (Huddinge kn)@X=17945678@Y=59217959@U=1@L=740000031@',
  'extId': '740000031',
  'routeIdx': 2,
  'lon': 17.945678,
  'lat': 59.217959,
  'depTime': '16:36:00',
  'depDate': '2025-01-07',
  'alighting': False,
  'minimumChangeDuration': 'PT10M'},
 {'name': 'Skövde Centralstation',
  'id': 'A=1@O=Skövde Centralstation@X=13853195@Y=58390898@U=1@L=740000008@',
  'extId': '740000008',
  'routeIdx': 3,
  'lon':

In [175]:
for stop in example_stops:
    print(stop['name'])


Stockholm Centralstation
Flemingsberg station (Huddinge kn)
Skövde Centralstation
Herrljunga station
Alingsås station
Göteborg Centralstation


In [176]:
[{stop.get("name"): stop.get("arrTime")} for stop in example_stops]

[{'Stockholm Centralstation': None},
 {'Flemingsberg station (Huddinge kn)': None},
 {'Skövde Centralstation': '18:39:00'},
 {'Herrljunga station': '19:11:00'},
 {'Alingsås station': '19:30:00'},
 {'Göteborg Centralstation': '20:00:00'}]

# Stop lookup API to find id

In [177]:
location = "göteborg"
url = f"https://api.resrobot.se/v2.1/location.name?input={location}&format=json&accessId={API_KEY}"
response = requests.get(url)
result = response.json()
result.keys()

dict_keys(['stopLocationOrCoordLocation', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'requestId'])

In [178]:
stop_locations = result["stopLocationOrCoordLocation"]
len(stop_locations)

10

In [179]:
stop_locations[0]

{'StopLocation': {'productAtStop': [{'icon': {'res': 'prod_gen'}, 'cls': '2'},
   {'icon': {'res': 'prod_gen'}, 'cls': '4'},
   {'icon': {'res': 'prod_gen'}, 'cls': '8'},
   {'icon': {'res': 'prod_gen'}, 'cls': '16'},
   {'icon': {'res': 'prod_gen'}, 'cls': '64'},
   {'icon': {'res': 'prod_gen'}, 'cls': '128'},
   {'icon': {'res': 'prod_gen'}, 'cls': '256'}],
  'timezoneOffset': 60,
  'id': 'A=1@O=GÖTEBORG@X=11973479@Y=57708895@U=1@L=740098001@B=1@p=1736224604@',
  'extId': '740098001',
  'name': 'GÖTEBORG',
  'lon': 11.973479,
  'lat': 57.708895,
  'weight': 15424,
  'products': 222,
  'meta': True,
  'minimumChangeDuration': 'PT26M'}}

In [180]:
stop_locations[0]["StopLocation"].keys()

dict_keys(['productAtStop', 'timezoneOffset', 'id', 'extId', 'name', 'lon', 'lat', 'weight', 'products', 'meta', 'minimumChangeDuration'])

In [181]:
stop_locations[0]["StopLocation"]["name"], stop_locations[0]["StopLocation"]["extId"]

('GÖTEBORG', '740098001')

In [182]:
print("Stops in Göteborg")
for stop in stop_locations:
    print(stop["StopLocation"]["name"], stop["StopLocation"]["extId"])

Stops in Göteborg
GÖTEBORG 740098001
Göteborg Centralstation 740000002
Göteborg Sävenäs lokstation 740016365
GÖTEBORG GAMLESTADEN 740098526
Göteborg Korsvägen 740015578
Göteborg Kungsportsplatsen 740016358
Göteborg Stenpiren 740072430
Göteborg Eketrägatan 740025624
Göteborg Vårväderstorget 740025707
Göteborg Axel Dahlströms torg 740025608


In [183]:
# :<50  -> left align in a 50-character wide field
print(f"{'Stop':<50} {'extId'}")
for stop_location in stop_locations:
    stop = stop_location["StopLocation"]
    print(f"{stop.get('name'):<50} {stop.get('extId')}")

Stop                                               extId
GÖTEBORG                                           740098001
Göteborg Centralstation                            740000002
Göteborg Sävenäs lokstation                        740016365
GÖTEBORG GAMLESTADEN                               740098526
Göteborg Korsvägen                                 740015578
Göteborg Kungsportsplatsen                         740016358
Göteborg Stenpiren                                 740072430
Göteborg Eketrägatan                               740025624
Göteborg Vårväderstorget                           740025707
Göteborg Axel Dahlströms torg                      740025608


In [184]:
def access_id_from_location(location):
    url = f"https://api.resrobot.se/v2.1/location.name?input={location}&format=json&accessId={API_KEY}"

    try:
        response = requests.get(url)
        result = response.json()

        print(f"{'Name':<50} extId")

        for stop in result.get("stopLocationOrCoordLocation"):
            stop_data = next(iter(stop.values()))

            # returns None if extId doesn't exist
            if stop_data.get("extId"):
                print(f"{stop_data['name']:<50} {stop_data['extId']}")

    except requests.exceptions.RequestException as err:
        print(f"Network or HTTP error: {err}")

access_id_from_location("malm")

Name                                               extId
Malm (Gullspång kn)                                740062061
Malmö Centralstation                               740000003
Malmö Triangeln station                            740001587
Malmö Hyllie station                               740001586
Malmö Svågertorp station                           740001546
Malmö Persborg station                             740001486
MALMÖ                                              740098548
Malmö Rosengård station                            740001621
Malmö Östervärn station                            740001483
Malmö Fosieby station                              740001553


In [185]:
access_id_from_location("onsala")

Name                                               extId
Onsala kyrka (Kungsbacka kn)                       740000092
Onsala vårdcentral (Kungsbacka kn)                 740061484


## Time tables
- [ResRobot Timetables](https://www.trafiklab.se/api/trafiklab-apis/resrobot-v21/timetables/)

Here I use pandas to read the json data, I'll leave it to you to make it into a function and find out relevant information

In [ ]:
# korsvägen
stop_id=740015578
url = f"https://api.resrobot.se/v2.1/departureBoard?id={stop_id}&format=json&accessId={API_KEY}"

response = requests.get(url)
results = response.json()
results.keys()

#  Make a function for doing get request for time tables API in resrobot as we didn't have a function in the lecture notes.


In [211]:
#  Make a function for doing get request for time tables API in resrobot as we didn't have a function in the lecture notes.

def get_departures(stop_id):
    url = f"https://api.resrobot.se/v2.1/departureBoard?id={stop_id}&format=json&accessId={API_KEY}"

    try:
        response = requests.get(url)
        response.raise_for_status()

        return response.json()
    except requests.exceptions.RequestException as err:
        print(f"Network or HTTP error: {err}")
        

In [ ]:
import pandas as pd 

df_timetable = pd.DataFrame(results["Departure"])
df_timetable.head()

,JourneyDetailRef,JourneyStatus,ProductAtStop,Product,Notes,name,type,stop,stopid,stopExtId,lon,lat,time,date,reachable,direction,directionFlag
0,{'ref': '1|257|0|1|7012025'},P,"{'icon': {'res': 'prod_gen'}, 'operatorInfo': ...","[{'icon': {'res': 'prod_gen'}, 'operatorInfo':...","{'Note': [{'value': 'Lag 2015:953 tillämpas', ...",Länstrafik - Buss X4,ST,Göteborg Korsvägen,A=1@O=Göteborg Korsvägen@X=11986918@Y=57696742...,740015578,11.986918,57.696742,16:18:00,2025-01-07,True,Höga hallar (Härryda kn),1
1,{'ref': '1|30103|8|1|7012025'},P,"{'icon': {'res': 'prod_gen'}, 'operatorInfo': ...","[{'icon': {'res': 'prod_gen'}, 'operatorInfo':...","{'Note': [{'value': 'Lag 2015:953 tillämpas', ...",Länstrafik - Spårväg 4,ST,Göteborg Korsvägen,A=1@O=Göteborg Korsvägen@X=11986918@Y=57696742...,740015578,11.986918,57.696742,16:18:00,2025-01-07,True,Angered centrum (Göteborg kn),1
2,{'ref': '1|115013|12|1|7012025'},P,"{'icon': {'res': 'prod_gen'}, 'operatorInfo': ...","[{'icon': {'res': 'prod_gen'}, 'operatorInfo':...","{'Note': [{'value': 'Lag 2015:953 tillämpas', ...",Länstrafik - Spårväg 2,ST,Göteborg Scandinavium,A=1@O=Göteborg Scandinavium@X=11985875@Y=57700...,740025681,11.985875,57.700445,16:18:00,2025-01-07,True,Mölndals Innerstad,1
3,{'ref': '1|154533|0|1|7012025'},P,"{'icon': {'res': 'prod_gen'}, 'operatorInfo': ...","[{'icon': {'res': 'prod_gen'}, 'operatorInfo':...","{'Note': [{'value': 'Lag 2015:953 tillämpas', ...",Länstrafik - Buss RÖD,ST,Göteborg Korsvägen,A=1@O=Göteborg Korsvägen@X=11986918@Y=57696742...,740015578,11.986918,57.696742,16:18:00,2025-01-07,True,Önneröd (Härryda kn),2
4,{'ref': '1|64617|1|1|7012025'},P,"{'icon': {'res': 'prod_gen'}, 'operatorInfo': ...","[{'icon': {'res': 'prod_gen'}, 'operatorInfo':...","{'Note': [{'value': 'Lag 2015:953 tillämpas', ...",Länstrafik - Spårväg 8,ST,Göteborg Korsvägen,A=1@O=Göteborg Korsvägen@X=11986918@Y=57696742...,740015578,11.986918,57.696742,16:19:00,2025-01-07,True,Frölunda torg (Göteborg kn),2


In [155]:
df_timetable.columns

Index(['JourneyDetailRef', 'JourneyStatus', 'ProductAtStop', 'Product',
       'Notes', 'name', 'type', 'stop', 'stopid', 'stopExtId', 'lon', 'lat',
       'time', 'date', 'reachable', 'direction', 'directionFlag'],
      dtype='object')

In [156]:
df_timetable[["name", "direction", "time"]]

,name,direction,time
0,Länstrafik - Buss X4,Höga hallar (Härryda kn),16:18:00
1,Länstrafik - Spårväg 4,Angered centrum (Göteborg kn),16:18:00
2,Länstrafik - Spårväg 2,Mölndals Innerstad,16:18:00
3,Länstrafik - Buss RÖD,Önneröd (Härryda kn),16:18:00
4,Länstrafik - Spårväg 8,Frölunda torg (Göteborg kn),16:19:00
...,...,...,...
222,Länstrafik - Buss RÖD,Lilla Varholmen (Göteborg kn),17:17:00
223,Länstrafik - Spårväg 5,Göteborg Varmfrontsgatan,17:17:00
224,Länstrafik - Spårväg 4,Angered centrum (Göteborg kn),17:18:00
225,Länstrafik - Spårväg 2,Mölndals Innerstad,17:18:00


In [157]:
df_timetable_cleaned = df_timetable[["name", "stop", "lon", "lat", "direction", "date", "time"]]
df_timetable_cleaned

,name,stop,lon,lat,direction,date,time
0,Länstrafik - Buss X4,Göteborg Korsvägen,11.986918,57.696742,Höga hallar (Härryda kn),2025-01-07,16:18:00
1,Länstrafik - Spårväg 4,Göteborg Korsvägen,11.986918,57.696742,Angered centrum (Göteborg kn),2025-01-07,16:18:00
2,Länstrafik - Spårväg 2,Göteborg Scandinavium,11.985875,57.700445,Mölndals Innerstad,2025-01-07,16:18:00
3,Länstrafik - Buss RÖD,Göteborg Korsvägen,11.986918,57.696742,Önneröd (Härryda kn),2025-01-07,16:18:00
4,Länstrafik - Spårväg 8,Göteborg Korsvägen,11.986918,57.696742,Frölunda torg (Göteborg kn),2025-01-07,16:19:00
...,...,...,...,...,...,...,...
222,Länstrafik - Buss RÖD,Göteborg Korsvägen,11.986918,57.696742,Lilla Varholmen (Göteborg kn),2025-01-07,17:17:00
223,Länstrafik - Spårväg 5,Göteborg Korsvägen,11.986918,57.696742,Göteborg Varmfrontsgatan,2025-01-07,17:17:00
224,Länstrafik - Spårväg 4,Göteborg Korsvägen,11.986918,57.696742,Angered centrum (Göteborg kn),2025-01-07,17:18:00
225,Länstrafik - Spårväg 2,Göteborg Scandinavium,11.985875,57.700445,Mölndals Innerstad,2025-01-07,17:18:00


In [158]:
df_timetable_cleaned["name"].value_counts()

name
Länstrafik - Spårväg 2    25
Länstrafik - Spårväg 8    24
Länstrafik - Spårväg 6    24
Länstrafik - Buss 18      24
Länstrafik - Buss X4      23
Länstrafik - Buss RÖD     19
Länstrafik - Buss 61      16
Länstrafik - Buss 63      14
Länstrafik - Spårväg 4    13
Länstrafik - Spårväg 5    12
Länstrafik - Buss 758     12
Länstrafik - Buss 100      8
Länstrafik - Buss 300      7
Länstrafik - Buss 101      3
Flygtransfer - Buss .      2
Expressbuss 611            1
Name: count, dtype: int64